# Clustering Crypto

In [158]:
# Initial imports
import requests
import pandas as pd
import matplotlib.pyplot as plt
import hvplot.pandas
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from pathlib import Path

### Fetching Cryptocurrency Data

In [180]:
# Alternatively, use the provided csv file:
cryptopath = Path("crypto_data.csv")

cryptodata = pd.read_csv(cryptopath)
cryptodata.head()

,Unnamed: 0,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
0,42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
1,365,365Coin,X11,True,PoW/PoS,NaN,2300000000
2,404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
3,611,SixEleven,SHA-256,True,PoW,NaN,611000
4,808,808,SHA-256,True,PoW/PoS,0.000000e+00,0


### Data Preprocessing

In [181]:
# Keep only necessary columns:
# 'CoinName','Algorithm','IsTrading','ProofType','TotalCoinsMined','TotalCoinSupply'
cryptodata.drop(columns=cryptodata.columns[0], axis=1, inplace=True)
cryptodata.head()

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
0,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
1,365Coin,X11,True,PoW/PoS,NaN,2300000000
2,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
3,SixEleven,SHA-256,True,PoW,NaN,611000
4,808,SHA-256,True,PoW/PoS,0.000000e+00,0


In [182]:
# Keep only cryptocurrencies that are trading
cryptodata.drop(cryptodata[cryptodata['IsTrading'] == False].index, inplace=True)
cryptodata.head()

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
0,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
1,365Coin,X11,True,PoW/PoS,NaN,2300000000
2,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
3,SixEleven,SHA-256,True,PoW,NaN,611000
4,808,SHA-256,True,PoW/PoS,0.000000e+00,0


In [183]:
# Keep only cryptocurrencies with a working algorithm
cryptodata = cryptodata.loc[cryptodata['Algorithm'] != 'N/A']
cryptodata.head()

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
0,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
1,365Coin,X11,True,PoW/PoS,NaN,2300000000
2,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
3,SixEleven,SHA-256,True,PoW,NaN,611000
4,808,SHA-256,True,PoW/PoS,0.000000e+00,0


In [184]:
# Remove the "IsTrading" column
cryptodata = cryptodata.drop(columns = 'IsTrading')
cryptodata.head()

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
0,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
1,365Coin,X11,PoW/PoS,NaN,2300000000
2,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
3,SixEleven,SHA-256,PoW,NaN,611000
4,808,SHA-256,PoW/PoS,0.000000e+00,0


In [185]:
# Remove rows with at least 1 null value
cryptodata = cryptodata.dropna()
cryptodata.head()

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
0,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
2,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
4,808,SHA-256,PoW/PoS,0.000000e+00,0
5,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
7,Bitcoin,SHA-256,PoW,1.792718e+07,21000000


In [186]:
# Remove rows with cryptocurrencies having no coins mined
cryptodata = cryptodata.loc[cryptodata['TotalCoinsMined'] != 0]
cryptodata.head()

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
0,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
2,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
5,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
7,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
8,Ethereum,Ethash,PoW,1.076842e+08,0


In [187]:
# Drop rows where there are 'N/A' text values
cryptodata = cryptodata.loc[cryptodata['ProofType'] != 'N/A']
cryptodata = cryptodata.loc[cryptodata['TotalCoinSupply'] != 'N/A']
cryptodata = cryptodata.loc[cryptodata['TotalCoinsMined'] != 'N/A']
cryptodata = cryptodata.loc[cryptodata['Algorithm'] != 'N/A']
cryptodata = cryptodata.loc[cryptodata['CoinName'] != 'N/A']
cryptodata.head()

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
0,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
2,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
5,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
7,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
8,Ethereum,Ethash,PoW,1.076842e+08,0


In [188]:
# Store the 'CoinName'column in its own DataFrame prior to dropping it from crypto_df
CoinName = pd.DataFrame(data=cryptodata['CoinName'])
CoinName.head()

,CoinName
0,42 Coin
2,404Coin
5,EliteCoin
7,Bitcoin
8,Ethereum


In [189]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm
cryptodata.index = cryptodata['CoinName']
cryptodata = cryptodata.drop(columns='CoinName')
cryptodata.index.names = ['']
cryptodata.head()

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
,,,,
42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
Bitcoin,SHA-256,PoW,1.792718e+07,21000000
Ethereum,Ethash,PoW,1.076842e+08,0


In [190]:
# Create dummy variables for text features
x = pd.get_dummies(data=cryptodata, columns=['Algorithm', 'ProofType', 'TotalCoinsMined', 'TotalCoinSupply'])

In [191]:
# Standardize data
cryptoscale = StandardScaler().fit_transform(x)
cryptoscale

array([[-0.0433555, -0.0433555, -0.0433555, ..., -0.0433555, -0.0433555,
        -0.0433555],
       [-0.0433555, -0.0433555, -0.0433555, ..., -0.0433555, -0.0433555,
        -0.0433555],
       [-0.0433555, -0.0433555, -0.0433555, ..., -0.0433555, -0.0433555,
        -0.0433555],
       ...,
       [-0.0433555, -0.0433555, -0.0433555, ..., -0.0433555, -0.0433555,
        -0.0433555],
       [-0.0433555, -0.0433555, -0.0433555, ..., -0.0433555, -0.0433555,
        -0.0433555],
       [-0.0433555, -0.0433555, -0.0433555, ..., -0.0433555, -0.0433555,
        -0.0433555]])

### Reducing Dimensions Using PCA

In [192]:
# Use PCA to reduce dimensions to 3 principal components
pca = PCA(n_components=3)
crypto_pca = pca.fit_transform(cryptoscale)

In [193]:
# Create a DataFrame with the principal components data
pca_df = pd.DataFrame(
    data=crypto_pca, columns=["PC 1", "PC 2", "PC 3"], index=cryptodata.index
)
pca_df.head()

,PC 1,PC 2,PC 3
,,,
42 Coin,-0.104771,-0.159797,-0.149739
404Coin,-0.171781,-0.182701,-0.041769
EliteCoin,0.109160,-0.221456,0.099227
Bitcoin,-0.138209,-0.058382,-0.137391
Ethereum,-0.287722,-0.161492,-0.104008


### Clustering Crytocurrencies Using K-Means

#### Find the Best Value for `k` Using the Elbow Curve

In [194]:
inertia = []
k = list(range(1, 11))

# Calculate the inertia for the range of k values
for i in k:
    km = KMeans(n_clusters=i, random_state=0)
    km.fit(pca_df)
    inertia.append(km.inertia_)

# Create the Elbow Curve using hvPlot
elbow_data = {"k": k, "inertia": inertia}
elbow = pd.DataFrame(elbow_data)
elbow.hvplot.line(x="k", y="inertia", xticks=k, title="Elbow Visualization", xlabel="Number of Clusters", ylabel="Inertia")

C:\Users\erfan\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:1036: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=3.
  warnings.warn(


:Curve   [k]   (inertia)

Running K-Means with `k=<your best value for k here>`

In [202]:
# Initialize the K-Means model
kmodel = KMeans(n_clusters=6, random_state=0)

# Fit the model
kmodel.fit(pca_df)

# Predict clusters
predictions = kmodel.predict(pca_df)

# Create a new DataFrame including predicted clusters and cryptocurrencies features
clustered_df = pd.DataFrame({
    "Algorithm": cryptodata.Algorithm,
    "ProofType": cryptodata.ProofType,
    "TotalCoinsMined": cryptodata.TotalCoinsMined,
    "TotalCoinSupply": cryptodata.TotalCoinSupply,
    "PC 1": pca_df['PC 1'],
    "PC 2": pca_df['PC 2'],
    "PC 3": pca_df['PC 3'],
    "CoinName": cryptodata.index,
    "Class": kmodel.labels_,
    },
)

clustered_df.head()

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,PC 1,PC 2,PC 3,CoinName,Class
,,,,,,,,,
42 Coin,Scrypt,PoW/PoS,4.199995e+01,42,-0.104771,-0.159797,-0.149739,42 Coin,5
404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000,-0.171781,-0.182701,-0.041769,404Coin,5
EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359,0.109160,-0.221456,0.099227,EliteCoin,5
Bitcoin,SHA-256,PoW,1.792718e+07,21000000,-0.138209,-0.058382,-0.137391,Bitcoin,5
Ethereum,Ethash,PoW,1.076842e+08,0,-0.287722,-0.161492,-0.104008,Ethereum,5


### Visualizing Results

#### Scatter Plot with Tradable Cryptocurrencies

In [204]:
# Scale data to create the scatter plot
clustered_df['TotalCoinsMined'] = clustered_df['TotalCoinsMined'].astype(float) / 1000000000
clustered_df['TotalCoinSupply'] = clustered_df['TotalCoinSupply'].astype(float) / 1000000000

In [206]:
# Plot the scatter with x="TotalCoinsMined" and y="TotalCoinSupply"
clustered_df.hvplot(
    kind="scatter", 
    x="TotalCoinsMined", 
    y="TotalCoinSupply", 
    c='Class', 
    hover_cols=['CoinName']
)

:Scatter   [TotalCoinsMined]   (TotalCoinSupply,Class,CoinName)

#### Table of Tradable Cryptocurrencies

In [207]:
# Table with tradable cryptos
columns = ['CoinName', 'Algorithm', 'ProofType', 'TotalCoinSupply', 'TotalCoinsMined', 'Class']
clustered_df.hvplot.table(columns)

:Table   [CoinName,Algorithm,ProofType,TotalCoinSupply,TotalCoinsMined,Class]

In [222]:
# Print the total number of tradable cryptocurrencies
print('The total number of tradable cryptocurrencies are:', len(clustered_df.index))

The total number of tradable cryptocurrencies are: 533
